In [4]:
import pandas as pd

df_mapped = pd.read_csv('./230817_intent_labeled_by_6.csv')

df_mapped = df_mapped.dropna(subset=['label'])

# Mapping function for '주문' label
def map_order_label(intent):
    if intent == '주문':
        return '주문'
    else:
        return intent

# Mapping function for '배송' label
def map_delivery_label(query, intent):
    if intent == '배송' or any(keyword in query for keyword in ['언제', '오늘', '언제쯤', '준비']):
        return '배송'
    else:
        return intent

# Mapping function for '매장' label
def map_store_label(query, intent):
    if intent == '매장' or '피팅룸' in query:
        return '매장'
    else:
        return intent

# Mapping function for 'AS' label
def map_as_label(query, intent):
    if intent == 'AS' or any(keyword in query for keyword in ['반품', '교환', '환불']):
        return 'AS'
    else:
        return intent

# Mapping function for '제품' label
def map_product_label(intent):
    if intent in ['제품_정보', '제품_재고']:
        return '제품'
    else:
        return intent

# Apply the mapping functions
df_mapped['intent'] = df_mapped['intent'].apply(map_order_label)
df_mapped['intent'] = df_mapped.apply(lambda row: map_delivery_label(row['query'], row['intent']), axis=1)
df_mapped['intent'] = df_mapped.apply(lambda row: map_store_label(row['query'], row['intent']), axis=1)
df_mapped['intent'] = df_mapped.apply(lambda row: map_as_label(row['query'], row['intent']), axis=1)
df_mapped['intent'] = df_mapped['intent'].apply(map_product_label)

# Step 2: Filter '제품' label queries based on the specified keywords
product_keywords = ['사이즈', '있나요', '제품', '없나요']
filtered_product_df = df_mapped[
    (df_mapped['intent'] == '제품') & 
    (df_mapped['query'].apply(lambda x: any(keyword in x for keyword in product_keywords)))
]

# Concatenate non-'제품' label rows and filtered '제품' label rows
final_filtered_df = pd.concat([
    df_mapped[df_mapped['intent'] != '제품'],
    filtered_product_df
])

# Step 3: Count the unique number of queries for each label
unique_counts_per_intent = final_filtered_df.groupby('intent')['query'].nunique()

unique_counts_per_intent

intent
AS    19533
매장     3733
배송    36570
제품    19233
주문      548
Name: query, dtype: int64

In [5]:
final_filtered_df.to_csv('./230817_intent_labeled_by_6.csv', index=False)